In [270]:
import requests
from bs4 import BeautifulSoup
import webbrowser
import time
import pandas as pd
# from general_utilities.web_scraping.parsers.parsing_methods import parse_html_tables

def __init__(self, **kwargs):
    self.api_payload = kwargs.get('api_payload', None)

def get_wonder_session():
    # Create a session object
    session = requests.Session()

    # Common headers to mimic a real browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }

    # Step 1: Access the initial URL
    initial_url = 'https://wonder.cdc.gov/mcd-icd10-provisional.html'
    response = session.get(initial_url, headers=headers)

    # Step 2: Extract the correct form for the "I Agree" button
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the correct form with the action URL '/controller/datarequest/D176'
    form = soup.find('form', action='/controller/datarequest/D176')
    action_url = form.get('action') if form else None

    if not action_url:
        print("Failed to find the agreement form.")
        exit()

    agree_url = 'https://wonder.cdc.gov' + action_url

    # Prepare the payload for the form submission
    payload = {}
    for input_tag in form.find_all('input'):
        name = input_tag.get('name')
        value = input_tag.get('value')
        if name:
            payload[name] = value

    response = session.post(agree_url, headers=headers, data=payload, allow_redirects=True)

    # Check if the agreement was successful
    if response.status_code != 200:
        print(f"Failed to submit the agreement form. Status code: {response.status_code}")
        print("Response:", response.text)
        exit()
    return session, response

def get_wonder_payload_response(api_payload=None):
    session, response = get_wonder_session()
    # Parse the response to get the next form or desired content
    soup = BeautifulSoup(response.content, 'html.parser')
    cookies = session.cookies
    cookie_header = '; '.join([f"{cookie.name}={cookie.value}" for cookie in cookies])

    # Locate the form with the action URL containing 'jsessionid'
    form = soup.find('form', action=lambda x: x and 'jsessionid' in x)
    if not form:
        print("Failed to find the next form with jsessionid.")
        exit()

    # Extract the action URL
    next_action_url = 'https://wonder.cdc.gov' + form['action']

    # Add cookies to your headers
    next_headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Referer': response.url,
        'Cookie': cookie_header  # Add the cookies here
    }

    # Use the provided payload or let the user submit the form
    if api_payload is None:
        api_payload = get_payload()
    
    # Make the POST request to the next action URL
    response = session.post(next_action_url, headers=next_headers, data=api_payload)
    return response

def fetch_and_parse_form(url: str):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find the first form (or adjust to find the specific form you need)
    form = soup.find('form')
    if not form:
        raise ValueError("No form found on the page.")
    
    # Extract form HTML
    form_html = str(form)
    
    return form_html





def get_payload(response):
    # Open the URL for the user to make their selections
    webbrowser.open(response.url)
    print("Please make your selections on the website and press the Send button.")

    # Wait for the user to press the Send button
    input("Press Enter after you have made your selections and pressed the Send button on the website...")

    # Simulate user payload capture (since we cannot capture actual user interaction)
    payload = {
        # Populate with data as if captured from user's selections
    }
    if not payload:
        payload = {'B_1': 'D176.V9-level1', 'B_2': 'D176.V15', 'B_3': '*None*', 'B_4': '*None*', 'B_5': '*None*', 'F_D176.V1': '*All*', 'F_D176.V10': '*All*', 'F_D176.V100': '*All*', 'F_D176.V13': '*All*', 'F_D176.V2': '*All*', 'F_D176.V25': '*All*', 'F_D176.V26': '*All*', 'F_D176.V27': '*All*', 'F_D176.V77': '*All*', 'F_D176.V79': '*All*', 'F_D176.V80': '*All*', 'F_D176.V9': '*All*', 'I_D176.V1': '*All* (All Dates)', 'I_D176.V10': '*All* (The United States)', 'I_D176.V100': '*All* (All Dates)', 'I_D176.V2': '*All* (All Causes of Death)', 'I_D176.V25': 'All Causes of Death', 'I_D176.V27': '*All* (The United States)', 'I_D176.V77': '*All* (The United States)', 'I_D176.V79': '*All* (The United States)', 'I_D176.V80': '*All* (The United States)', 'I_D176.V9': '*All* (The United States)', 'L_D176.V15': '*All*', 'L_D176.V16': '*All*', 'M_1': 'D176.M1', 'M_2': 'D176.M2', 'M_3': 'D176.M3', 'M_31': 'D176.M31', 'M_32': 'D176.M32', 'M_9': 'D176.M9', 'O_MMWR': 'false', 'O_V100_fmode': 'freg', 'O_V10_fmode': 'freg', 'O_V13_fmode': 'fadv', 'O_V15_fmode': 'fadv', 'O_V16_fmode': 'fadv', 'O_V1_fmode': 'freg', 'O_V25_fmode': 'freg', 'O_V26_fmode': 'fadv', 'O_V27_fmode': 'freg', 'O_V2_fmode': 'freg', 'O_V77_fmode': 'freg', 'O_V79_fmode': 'freg', 'O_V80_fmode': 'freg', 'O_V9_fmode': 'freg', 'O_aar': 'aar_std', 'O_aar_CI': 'true', 'O_aar_SE': 'true', 'O_aar_enable': 'true', 'O_aar_pop': '0000', 'O_age': 'D176.V5', 'O_dates': 'YEAR', 'O_death_location': 'D176.V79', 'O_death_urban': 'D176.V89', 'O_javascript': 'on', 'O_location': 'D176.V9', 'O_mcd': 'D176.V13', 'O_oc-sect1-request': 'close', 'O_precision': '1', 'O_race': 'D176.V42', 'O_rate_per': '100000', 'O_show_totals': 'true', 'O_timeout': '600', 'O_title': '', 'O_ucd': 'D176.V2', 'O_urban': 'D176.V19', 'VM_D176.M6_D176.V10': '', 'VM_D176.M6_D176.V17': '*All*', 'VM_D176.M6_D176.V1_S': '*All*', 'VM_D176.M6_D176.V42': '*All*', 'VM_D176.M6_D176.V7': '*All*', 'V_D176.V1': '', 'V_D176.V10': '', 'V_D176.V100': '', 'V_D176.V11': '*All*', 'V_D176.V12': '*All*', 'V_D176.V13': '', 'V_D176.V13_AND': '', 'V_D176.V15': '', 'V_D176.V15_AND': '', 'V_D176.V16': '', 'V_D176.V16_AND': '', 'V_D176.V17': '*All*', 'V_D176.V19': '*All*', 'V_D176.V2': '', 'V_D176.V20': '*All*', 'V_D176.V21': '*All*', 'V_D176.V22': '*All*', 'V_D176.V23': '*All*', 'V_D176.V25': '', 'V_D176.V26': '', 'V_D176.V26_AND': '', 'V_D176.V27': '', 'V_D176.V4': '*All*', 'V_D176.V42': '*All*', 'V_D176.V43': '*All*', 'V_D176.V44': '*All*', 'V_D176.V5': '*All*', 'V_D176.V51': '*All*', 'V_D176.V52': '*All*', 'V_D176.V6': '00', 'V_D176.V7': '*All*', 'V_D176.V77': '', 'V_D176.V79': '', 'V_D176.V80': '', 'V_D176.V81': '*All*', 'V_D176.V89': '*All*', 'V_D176.V9': '', 'action-Send': 'Send', 'dataset_code': 'D176', 'dataset_label': 'Provisional Mortality Statistics, 2018 through Last Week', 'dataset_vintage': 'June 15, 2024 as of June 23, 2024', 'finder-stage-D176.V1': 'codeset', 'finder-stage-D176.V10': 'codeset', 'finder-stage-D176.V100': 'codeset', 'finder-stage-D176.V13': 'codeset', 'finder-stage-D176.V15': '', 'finder-stage-D176.V16': '', 'finder-stage-D176.V2': 'codeset', 'finder-stage-D176.V25': 'codeset', 'finder-stage-D176.V26': 'codeset', 'finder-stage-D176.V27': 'codeset', 'finder-stage-D176.V77': 'codeset', 'finder-stage-D176.V79': 'codeset', 'finder-stage-D176.V80': 'codeset', 'finder-stage-D176.V9': 'codeset', 'saved_id': '', 'stage': 'request'}
    return payload

def clean_tables(tables):
        cleaned_tables = []
        for table in tables:
            if not cleaned_tables:
                cleaned_tables.append(table)
                continue
            for cleaned_table in cleaned_tables:
                if (table.find('tbody')==cleaned_table.find('tbody')) and (table.find('thead')==cleaned_table.find('thead')):
                    keeper = False
                    break
                keeper = True
            if keeper:
                cleaned_tables.append(table)
        return cleaned_tables

def get_wonder_tables(api_payload=None):
    response = get_wonder_payload_response(api_payload)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    keeper = False
    # Find all tables in the HTML content
    tables = [x for x in soup.find_all('table') if x.find('tbody')]
    tables = clean_tables(tables)
    return tables

def extract_wonder_table_data(table) -> pd.DataFrame:
    # Extract headers from the table
    headers = []
    for th in table.thead.find_all('th'):
        header = th.get_text(strip=True)
        if header and header not in headers:  # Only add non-empty headers
            headers.append(header)
    print(f'there are {len(headers)} headers: {headers}')

    body = table.find('tbody')
    rows = []
    for row in body.find_all('tr'):
        row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
        rows.append(row_data)
    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=headers)
    return df

# def __main__(api_payload=None):
#     tables = get_wonder_tables(api_payload)
#     dfs = []
#     for table in tables:
#         df = extract_wonder_table_data(table)
#         print(df.head())
#         dfs.append(df)
#     return dfs

# if __name__ == '__main__':
#     __main__()

KeyboardInterrupt: 

In [261]:
target = tables[0]
    # get the title div
title_div = target.find('div', class_='title').find('h1').text
subtitle_div = target.find('div', class_='title').find('h2').text
header = target.find('thead')
headers = []
for head in header.find_all('th'):
    if head.text.strip() not in headers and head.text.strip() != '':
        headers.append(head.text.strip())
headers

['Residence State',
 'Ten-Year Age Groups',
 'UCD - Injury Intent',
 'Year',
 'Deaths']

In [269]:
print(payload)

{'B_1': 'D176.V9-level1', 'B_2': 'D176.V15', 'B_3': '*None*', 'B_4': '*None*', 'B_5': '*None*', 'F_D176.V1': '*All*', 'F_D176.V10': '*All*', 'F_D176.V100': '*All*', 'F_D176.V13': '*All*', 'F_D176.V2': '*All*', 'F_D176.V25': '*All*', 'F_D176.V26': '*All*', 'F_D176.V27': '*All*', 'F_D176.V77': '*All*', 'F_D176.V79': '*All*', 'F_D176.V80': '*All*', 'F_D176.V9': '*All*', 'I_D176.V1': '*All* (All Dates)', 'I_D176.V10': '*All* (The United States)', 'I_D176.V100': '*All* (All Dates)', 'I_D176.V2': '*All* (All Causes of Death)', 'I_D176.V25': 'All Causes of Death', 'I_D176.V27': '*All* (The United States)', 'I_D176.V77': '*All* (The United States)', 'I_D176.V79': '*All* (The United States)', 'I_D176.V80': '*All* (The United States)', 'I_D176.V9': '*All* (The United States)', 'L_D176.V15': '*All*', 'L_D176.V16': '*All*', 'M_1': 'D176.M1', 'M_2': 'D176.M2', 'M_3': 'D176.M3', 'M_31': 'D176.M31', 'M_32': 'D176.M32', 'M_9': 'D176.M9', 'O_MMWR': 'false', 'O_V100_fmode': 'freg', 'O_V10_fmode': 'freg'

In [263]:


body = target.find('tbody')
rows = []
for row in body.find_all('tr'):
    row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
    rows.append(row_data)
len(rows)
rows[0:1]

[['Arizona (04)', '25-34 years', 'Unintentional', '2020', '16']]

In [264]:
df = pd.DataFrame(rows, columns=headers)
df.insert(0, 'title', title_div)
df.insert(1, 'subtitle', subtitle_div)
df

,title,subtitle,Residence State,Ten-Year Age Groups,UCD - Injury Intent,Year,Deaths
0,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Arizona (04),25-34 years,Unintentional,2020,16
1,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Arizona (04),25-34 years,Unintentional,2021,19
2,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Arizona (04),25-34 years,Unintentional,2022,20
3,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Arizona (04),25-34 years,Unintentional,2023 (provisional),16
4,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",California (06),25-34 years,Unintentional,2018,"1,898"
...,...,...,...,...,...,...,...
57,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Texas (48),25-34 years,Unintentional,2022,17
58,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Texas (48),25-34 years,Unintentional,2023 (provisional),14
59,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Texas (48),25-34 years,"Non-Injury, no intent classified",2023 (provisional),12
60,"Provisional Mortality Statistics, 2018 through...","Deaths occurring through June 15, 2024 as of J...",Washington (53),25-34 years,Unintentional,2019,14


In [236]:


# Default payload
default_payload = {
  "B_1": "D176.V9-level1",
  "B_2": "D176.V5",
  "B_3": "D176.V22",
  "B_4": "D176.V1-level1",
  "B_5": "*None*",
  "F_D176.V1": "*All*",
  "F_D176.V10": "*All*",
  "F_D176.V100": "*All*",
  "F_D176.V13": "*All*",
  "F_D176.V2": "*All*",
  "F_D176.V25": "*All*",
  "F_D176.V26": "*All*",
  "F_D176.V27": "*All*",
  "F_D176.V77": "*All*",
  "F_D176.V79": "06",
  "F_D176.V80": "*All*",
  "F_D176.V9": "*All*",
  "I_D176.V1": "*All* (All Dates)",
  "I_D176.V10": "*All* (The United States)",
  "I_D176.V100": "*All* (All Dates)",
  "I_D176.V2": "*All* (All Causes of Death)",
  "I_D176.V25": "All Causes of Death",
  "I_D176.V27": "*All* (The United States)",
  "I_D176.V77": "*All* (The United States)",
  "I_D176.V79": "06 (California)",
  "I_D176.V80": "*All* (The United States)",
  "I_D176.V9": "*All* (The United States)",
  "L_D176.V15": "*All*",
  "L_D176.V16": "*All*",
  "M_1": "D176.M1",
  "M_2": "D176.M2",
  "M_3": "D176.M3",
  "O_MMWR": "false",
  "O_V100_fmode": "freg",
  "O_V10_fmode": "freg",
  "O_V13_fmode": "fadv",
  "O_V15_fmode": "fadv",
  "O_V16_fmode": "fadv",
  "O_V1_fmode": "freg",
  "O_V25_fmode": "freg",
  "O_V26_fmode": "fadv",
  "O_V27_fmode": "freg",
  "O_V2_fmode": "freg",
  "O_V77_fmode": "freg",
  "O_V79_fmode": "freg",
  "O_V80_fmode": "freg",
  "O_V9_fmode": "freg",
  "O_aar": "aar_none",
  "O_aar_pop": "0000",
  "O_age": "D176.V5",
  "O_dates": "YEAR",
  "O_death_location": "D176.V79",
  "O_death_urban": "D176.V89",
  "O_javascript": "on",
  "O_location": "D176.V9",
  "O_mcd": "D176.V13",
  "O_oc-sect1-request": "close",
  "O_precision": "1",
  "O_race": "D176.V42",
  "O_rate_per": "100000",
  "O_show_totals": "true",
  "O_timeout": "600",
  "O_title": "",
  "O_ucd": "D176.V2",
  "O_urban": "D176.V19",
  "VM_D176.M6_D176.V10": "",
  "VM_D176.M6_D176.V17": "*All*",
  "VM_D176.M6_D176.V1_S": "*All*",
  "VM_D176.M6_D176.V42": "*All*",
  "VM_D176.M6_D176.V7": "*All*",
  "V_D176.V1": "",
  "V_D176.V10": "",
  "V_D176.V100": "",
  "V_D176.V11": "*All*",
  "V_D176.V12": "*All*",
  "V_D176.V13": "",
  "V_D176.V13_AND": "",
  "V_D176.V15": "",
  "V_D176.V15_AND": "",
  "V_D176.V16": "",
  "V_D176.V16_AND": "",
  "V_D176.V17": "*All*",
  "V_D176.V19": "*All*",
  "V_D176.V2": "",
  "V_D176.V20": "*All*",
  "V_D176.V21": "*All*",
  "V_D176.V22": "*All*",
  "V_D176.V23": "*All*",
  "V_D176.V25": "",
  "V_D176.V26": "",
  "V_D176.V26_AND": "",
  "V_D176.V27": "",
  "V_D176.V4": "*All*",
  "V_D176.V42": "*All*",
  "V_D176.V43": "*All*",
  "V_D176.V44": "*All*",
  "V_D176.V5": "25-34",
  "V_D176.V51": "*All*",
  "V_D176.V52": "*All*",
  "V_D176.V6": "00",
  "V_D176.V7": "*All*",
  "V_D176.V77": "",
  "V_D176.V79": "",
  "V_D176.V80": "",
  "V_D176.V81": "*All*",
  "V_D176.V89": "*All*",
  "V_D176.V9": "",
  "action-Send": "Send",
  "dataset_code": "D176",
  "dataset_label": "Provisional Mortality Statistics, 2018 through Last Week",
  "dataset_vintage": "June 15, 2024 as of June 23, 2024",
  "finder-stage-D176.V1": "codeset",
  "finder-stage-D176.V10": "codeset",
  "finder-stage-D176.V100": "codeset",
  "finder-stage-D176.V13": "codeset",
  "finder-stage-D176.V15": "",
  "finder-stage-D176.V16": "",
  "finder-stage-D176.V2": "codeset",
  "finder-stage-D176.V25": "codeset",
  "finder-stage-D176.V26": "codeset",
  "finder-stage-D176.V27": "codeset",
  "finder-stage-D176.V77": "codeset",
  "finder-stage-D176.V79": "codeset",
  "finder-stage-D176.V80": "codeset",
  "finder-stage-D176.V9": "codeset",
  "saved_id": "",
  "stage": "request"
}


# Get the tables
# response = get_wonder(default_payload)

soup = BeautifulSoup(response.content, 'html.parser')
## now get all the tables that have a tbody
tables = [x for x in soup.find_all('table') if x.find('tbody')]
dfs=[]
for table in tables:
    df = wonder_table_to_df(table)


AttributeError: 'NoneType' object has no attribute 'find'

In [234]:
wonder_table_to_df(table)

AttributeError: 'NoneType' object has no attribute 'find'

In [193]:

table = tables[0]
headers = [x.text for x in table.find_all('th')]
data = []
for row in table.find_all('tr'):
    data.append([x.text for x in row.find_all('td')])


ValueError: 275 columns passed, passed data had 150 columns

In [191]:
url = 'https://wonder.cdc.gov/mcd-icd10-provisional.html'
def fetch_and_parse_form(url: str):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find the first form (or adjust to find the specific form you need)
    form = soup.find('form')
    if not form:
        raise ValueError("No form found on the page.")
    
    # Extract form HTML
    form_html = str(form)
    
    return form_html

['Washington (53)', '25-34 years', 'Unintentional', '2021', '10']